TO DO:
- look into migration dfs uses
- double check work for notebook!
- export all csvs!
- have a lil party

COMPLETED:
- Write a read me
- Look into mixed data type columns in birding data - will read in as str due to "X" in some rows
- Look into “sensitive species” list
- Figure out how to read in specific tabs in the xlsx files for FIA
- Separate observation date into year columm to group by 
- Clean tables to drop extraneous columns
- Figure out what years to restrict data
- compile list of birds into one df
- Merge birds table with USFWS table to get specific region & download doc off data.gov website
- Separate birds tables into USFWS regions & not
- create new column for seasons of birding
- Value counts on bird species to get top 10
- Create subset of birds in top 10 & group by year
- Replace all USFWS with IBA
- Read in IBA df & join with non-null IBA
- home in on species of interest
- Create subsets of 50-75 percentile birds
- Get count of unique bird observer ids per year
- Get stat for count of birds per unique bird observer id per year

General notes:
- Years to look at: 2007-2016
- Group by Important Bird Areas (IBA)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
# import matplotlib.pyplot as plt
# %matplotlib inline
# import folium

In [ ]:
# reading in IBA map
iba_map = gpd.read_file('../data/iba_map.geojson')
iba_map = iba_map[['SITE_ID', 'SITE_NAME', 'STATE', 'LATITUDE', 'LONGITUDE', 'geometry']]
iba_map.columns = ['iba_code', 'iba_name', 'state', 'latitude', 'longitude', 'geometry']
iba_map = iba_map.loc[iba_map.state == 'Tennessee']

# converting iba code column to int to get rid of '.0'
iba_map = iba_map.astype({'iba_code':int})

In [ ]:
# iba code & name subset
iba_names = iba_map[['iba_code', 'iba_name']]
iba_names.head(3)

In [ ]:
# reading in birds, 2007 to 2016
birds = pd.read_csv('../data/eBird_2007_to_2016_TN/eBird_2007_to_2016_TN.txt', sep='\t')

# cleaning birds df
birds = birds[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'STATE', 'COUNTY', 'IBA CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
birds.columns = ['global_unique_identifier', 'observation_date', 'category', 'common_name', 'scientific_name', 'state', 'county', 'iba_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']
birds = birds.loc[birds.category == 'species']
# creating additional column for observation year
birds['observation_year'] = [x[:4] for x in birds.observation_date]
birds = birds.astype({'observation_year':int})
birds.shape

In [ ]:
# reading in sensitive species list
sensitive_species = pd.read_csv('../data/sensitive_species_2000_2020_TN.txt', sep='\t')
sensitive_species = sensitive_species[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'STATE', 'COUNTY', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
sensitive_species.columns = ['global_unique_identifier', 'observation_date', 'category', 'common_name', 'scientific_name', 'state', 'county', 'iba_code', 'bcr_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']

# creating additional column for observation year
sensitive_species['observation_year'] = [x[:4] for x in sensitive_species.observation_date]
sensitive_species = sensitive_species.astype({'observation_year':int})

# restricting to relevant years
sensitive_species = sensitive_species.loc[(sensitive_species.observation_year < 2017)&(sensitive_species.observation_year > 2006)].sort_values('observation_year').reset_index(drop=True)

In [ ]:
# reading in USFWS codes
usfws_codes = pd.read_csv('../data/eBird_2007_to_2016_TN/USFWSCodes.txt', sep='\t')

# cleaning usfws codes df
usfws_codes = usfws_codes.reset_index()
usfws_codes['usfws_name'] = np.where(usfws_codes['USFWS NAME'].isnull(), usfws_codes['USFWS CODE'], usfws_codes['USFWS NAME'])
usfws_codes = usfws_codes[['index','usfws_name']]
usfws_codes.columns = ['usfws_code', 'usfws_name']
usfws_codes

### Adding columns for grouping

In [ ]:
# creating seasons column: spring migration, fall migration, and offseason

# creating month column
birds['observation_month'] = [x[5:7] for x in birds.observation_date]
birds = birds.astype({'observation_month':int})

# spring: March to May
# fall: Sept to Oct

# # categorizing season based on date
for index, row in birds.iterrows():
    if (row['observation_month'] <= 5)&(row['observation_month'] >= 3):
        birds.loc[index, 'season'] = 'spring migration'
    elif (row['observation_month'] <= 10)&(row['observation_month'] >= 9):
        birds.loc[index, 'season'] = 'fall migration'
    else:
        birds.loc[index, 'season'] = 'offseason'

In [ ]:
birds_by_season = birds.groupby('season').count()[['global_unique_identifier']]
birds_by_season.columns = ['count']
birds_by_season = birds_by_season.reset_index()
birds_by_season

In [ ]:
# subset of fall & spring migration
fall_birds = birds.loc[birds.season == 'fall migration']
spring_birds = birds.loc[birds.season == 'spring migration']

In [ ]:
# more to do with migration?

## Species of interest: Bald Eagle, Sandhill Crane, Baltimore Oriole, Indigo Bunting, Ruby-throated Hummingbird, Yellow Warbler, Summer Tanager

In [ ]:
# creating birds of interest df
birds_of_interest = birds.loc[birds.common_name.isin(['Sandhill Crane', 'Bald Eagle', 'Baltimore Oriole', 'Indigo Bunting', 'Ruby-throated Hummingbird', 'Yellow Warbler', 'Summer Tanager'])]

# creating separate count list
birds_of_interest_count = birds_of_interest.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
birds_of_interest_count.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds =  birds_of_interest_count.groupby('observation_year').sum()[['count']].reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']

# merging back
birds_of_interest_count = pd.merge(birds_of_interest_count, total_birds, on='observation_year', how='inner')

# adding perc column
birds_of_interest_count['perc_total_sightings'] = round(birds_of_interest_count['count']/birds_of_interest_count.total_count_by_year*100,2)

# total_birds
birds_of_interest_count

In [ ]:
# creating geometry column for birds of interest
birds_of_interest['geometry'] = birds_of_interest.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)
birds_of_interest

## Davidson County birds (downtown Nashville)

In [ ]:
# interested in 50-75 percentile
nashville_birds = birds.loc[birds.county == 'Davidson']
nashville_birds_count = nashville_birds.common_name.value_counts().reset_index()
nashville_birds_count = nashville_birds_count.loc[(nashville_birds_count['count']>33)&(nashville_birds_count['count']<120)]

# creating list of populations of focus
nashville_birds_list = nashville_birds_count.common_name.to_list()

# saving back to nashville birds
nashville_birds = nashville_birds.loc[nashville_birds.common_name.isin(nashville_birds_list)]

# grouping by year & common name for count
nash_birds_count = nashville_birds.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
nash_birds_count.columns = ['observation_year', 'common_name', 'count']

# adding geometry column for all nash bird sightings
nashville_birds['geometry'] = nashville_birds.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)

In [ ]:
nashville_birds_count.describe()

## IBA

In [ ]:
# creating subset of birds sighted within IBAs
iba_birds = birds.loc[~birds.iba_code.isna()]
iba_birds['iba_code'] = [x[6:10] for x in iba_birds.iba_code]
iba_birds = iba_birds.astype({'iba_code':int})
iba_birds = pd.merge(iba_birds, iba_names, on='iba_code', how='left')
iba_birds = iba_birds[['global_unique_identifier', 'observation_date', 'category', 'common_name', 'scientific_name', 'state', 'county', 'iba_code', 'lat', 'long', 'observer_id', 'trip_comments', 'observation_year', 'iba_name']]
iba_birds.head(3)

In [ ]:
# creating separate list of sightings outside of IBA regions
birds_outside_iba = birds.loc[birds.iba_code.isna()]
birds_outside_iba.drop(columns=['usfws_code', 'iba_code'])
birds_outside_iba['within_iba'] = False
birds_outside_iba = birds_outside_iba[['global_unique_identifier', 'observation_date', 'category', 'common_name', 'scientific_name', 'state', 'county', 'lat', 'long', 'observer_id', 'trip_comments', 'observation_year', 'within_iba']]

birds_outside_iba.head()

## USFWS

In [ ]:
# creating subset of birds sighted within usfws polygon:
birds_usfws = birds.loc[~birds.usfws_code.isna()]

# joining birds_usfws with usfws codes
birds_usfws = pd.merge(birds_usfws, usfws_codes, on='usfws_code', how='left')

birds_usfws.head(3)

In [ ]:
# creating separate list of sightings outside of USFWS regions
birds_outside_usfws = birds.loc[birds.usfws_code.isna()]
birds_outside_usfws.drop(columns=['usfws_code', 'iba_code'])
birds_outside_usfws['within_park'] = False

birds_outside_usfws.head()

## EDA

## Top 10 lists

#### with IBA code

In [ ]:
# creating subset called outside iba top 10, grouping by year & common name
outside_iba_top10 = birds_outside_iba.loc[birds_outside_iba.common_name.isin(['Northern Cardinal', 'American Crow', 'Carolina Chickadee', 'Blue Jay', 'Carolina Wren', 'Tufted Titmouse', 'Mourning Dove', 'American Robin', 'Red-bellied Woodpecker', 'Northern Mockingbird'])]
outside_iba_top10 = outside_iba_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
outside_iba_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds = outside_iba_top10.groupby('observation_year').sum()[['count']]
total_birds = total_birds.reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']
# merging back
outside_iba_top10 = pd.merge(outside_iba_top10, total_birds, on='observation_year', how='inner')

# adding perc column
outside_iba_top10['perc_total_sightings'] = round(outside_iba_top10['count']/outside_iba_top10.total_count_by_year*100,2)

outside_iba_top10

In [ ]:
# creating subset called outside iba next top 10, grouping by year & common name
outside_iba_next_top10 = birds_outside_iba.loc[birds_outside_iba.common_name.isin(['American Goldfinch', 'Downy Woodpecker', 'Eastern Towhee', 'European Starling', 'Eastern Bluebird', 'Song Sparrow', 'Canada Goose', 'Great Blue Heron', 'House Finch', 'White-breasted Nuthatch'])]
outside_iba_next_top10 = outside_iba_next_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
outside_iba_next_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds = outside_iba_next_top10.groupby('observation_year').sum()[['count']]
total_birds = total_birds.reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']
# merging back
outside_iba_next_top10 = pd.merge(outside_iba_next_top10, total_birds, on='observation_year', how='inner')

# adding perc column
outside_iba_next_top10['perc_total_sightings'] = round(outside_iba_next_top10['count']/outside_iba_next_top10.total_count_by_year*100,2)

outside_iba_next_top10

In [ ]:
# creating subset called iba top 10, grouping by year & common name
iba_top10 = iba_birds.loc[iba_birds.common_name.isin(['Great Blue Heron', 'American Crow', 'Northern Cardinal', 'Downy Woodpecker', 'American Robin', 'Carolina Wren', 'Carolina Chickadee', 'Red-bellied Woodpecker', 'Blue Jay', 'Tufted Titmouse'])]
iba_top10 = iba_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
iba_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds = iba_top10.groupby('observation_year').sum()[['count']]
total_birds = total_birds.reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']
# merging back
iba_top10 = pd.merge(iba_top10, total_birds, on='observation_year', how='inner')

# adding perc column
iba_top10['perc_total_sightings'] = round(iba_top10['count']/iba_top10.total_count_by_year*100,2)

iba_top10

In [ ]:
# creating subset called iba next top 10 (11-20), grouping by year & common name
iba_next_top10 = iba_birds.loc[iba_birds.common_name.isin(['Pileated Woodpecker', 'Canada Goose', 'American Goldfinch', 'Mourning Dove', 'Eastern Towhee', 'Eastern Bluebird', 'White-breasted Nuthatch', 'Mallard', 'Indigo Bunting', 'Red-eyed Vireo'])]
iba_next_top10 = iba_next_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
iba_next_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds = iba_next_top10.groupby('observation_year').sum()[['count']]
total_birds = total_birds.reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']
# merging back
iba_next_top10 = pd.merge(iba_next_top10, total_birds, on='observation_year', how='inner')

# adding perc column
iba_next_top10['perc_total_sightings'] = round(iba_next_top10['count']/iba_next_top10.total_count_by_year*100,2)
iba_next_top10

#### list of iba birds for map

In [ ]:
# creating subsets of iba birds with all sightings of top 10

iba_top10_list = iba_birds.loc[iba_birds.common_name.isin(['Great Blue Heron', 'American Crow', 'Northern Cardinal', 'Downy Woodpecker', 'American Robin', 'Carolina Wren', 'Carolina Chickadee', 'Red-bellied Woodpecker', 'Blue Jay', 'Tufted Titmouse'])]
iba_top10_list['geometry'] = iba_top10_list.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)

# creating subsets of iba birds with all sightings of next top 10

iba_next_top10_list = iba_birds.loc[iba_birds.common_name.isin(['Pileated Woodpecker', 'Canada Goose', 'American Goldfinch', 'Mourning Dove', 'Eastern Towhee', 'Eastern Bluebird', 'White-breasted Nuthatch', 'Mallard', 'Indigo Bunting', 'Red-eyed Vireo'])]
iba_next_top10_list['geometry'] = iba_next_top10_list.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)
# joining two dfs into top 20 list
iba_top20 = pd.concat([iba_top10_list, iba_next_top10_list])

#### list of birds outside iba for map

In [ ]:
# creating subsets of outside iba birds with all sightings of top 10

outside_iba_top10_list = birds_outside_iba.loc[birds_outside_iba.common_name.isin(['Northern Cardinal', 'American Crow', 'Carolina Chickadee', 'Blue Jay', 'Carolina Wren', 'Tufted Titmouse', 'Mourning Dove', 'American Robin', 'Red-bellied Woodpecker', 'Northern Mockingbird'])]
outside_iba_top10_list['geometry'] = outside_iba_top10_list.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)

# creating subsets of outside iba birds with all sightings of next top 10

outside_iba_next_top10_list = birds_outside_iba.loc[birds_outside_iba.common_name.isin(['American Goldfinch', 'Downy Woodpecker', 'Eastern Towhee', 'European Starling', 'Eastern Bluebird', 'Song Sparrow', 'Canada Goose', 'Great Blue Heron', 'House Finch', 'White-breasted Nuthatch'])]
outside_iba_next_top10_list['geometry'] = outside_iba_next_top10_list.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)

# joining two dfs into top 20 list
outside_iba_top20 = pd.concat([outside_iba_top10_list, outside_iba_next_top10_list])

#### with USFWS code

In [ ]:
# # creating subset called outside usfws top 10, grouping by year & common name
# outside_usfws_top10 = birds_outside_usfws.loc[birds_outside_usfws.common_name.isin(['Northern Cardinal', 'American Crow', 'Carolina Chickadee', 'Blue Jay', 'Carolina Wren', 'Tufted Titmouse', 'Mourning Dove', 'American Robin', 'Red-bellied Woodpecker', 'Northern Mockingbird'])]
# outside_usfws_top10 = outside_usfws_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
# outside_usfws_top10.columns = ['observation_year', 'common_name', 'count']

# # adding total count per year as a column
# total_birds = outside_usfws_top10.groupby('observation_year').sum()[['count']]
# total_birds = total_birds.reset_index()
# total_birds.columns=['observation_year', 'total_count_by_year']
# # merging back
# outside_usfws_top10 = pd.merge(outside_usfws_top10, total_birds, on='observation_year', how='inner')

# # adding perc column
# outside_usfws_top10['perc_total_sightings'] = round(outside_usfws_top10['count']/outside_usfws_top10.total_count_by_year*100,2)

# outside_usfws_top10
# # .loc[outside_usfws_top10.common_name == 'Northern Mockingbird']

In [ ]:
# # creating subset called outside usfws next top 10, grouping by year & common name
# outside_usfws_next_top10 = birds_outside_usfws.loc[birds_outside_usfws.common_name.isin(['American Goldfinch', 'Downy Woodpecker', 'Eastern Towhee', 'European Starling', 'Eastern Bluebird', 'Song Sparrow', 'Canada Goose', 'Great Blue Heron', 'House Finch', 'White-breasted Nuthatch'])]
# outside_usfws_next_top10 = outside_usfws_next_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
# outside_usfws_next_top10.columns = ['observation_year', 'common_name', 'count']

# # adding total count per year as a column
# total_birds = outside_usfws_next_top10.groupby('observation_year').sum()[['count']]
# total_birds = total_birds.reset_index()
# total_birds.columns=['observation_year', 'total_count_by_year']
# # merging back
# outside_usfws_next_top10 = pd.merge(outside_usfws_next_top10, total_birds, on='observation_year', how='inner')

# # adding perc column
# outside_usfws_next_top10['perc_total_sightings'] = round(outside_usfws_next_top10['count']/outside_usfws_next_top10.total_count_by_year*100,2)

# outside_usfws_next_top10

In [ ]:
# # creating subset called usfws top 10, grouping by year & common name
# usfws_top10 = birds_usfws.loc[birds_usfws.common_name.isin(['Great Blue Heron', 'American Crow', 'Northern Cardinal', 'Killdeer', 'Canada Goose', 'Carolina Wren', 'Mallard', 'Red-bellied Woodpecker', 'Bald Eagle', 'Tufted Titmouse'])]
# usfws_top10 = usfws_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
# usfws_top10.columns = ['observation_year', 'common_name', 'count']

# # adding total count per year as a column
# total_birds = usfws_top10.groupby('observation_year').sum()[['count']]
# total_birds = total_birds.reset_index()
# total_birds.columns=['observation_year', 'total_count_by_year']
# # merging back
# usfws_top10 = pd.merge(usfws_top10, total_birds, on='observation_year', how='inner')

# # adding perc column
# usfws_top10['perc_total_sightings'] = round(usfws_top10['count']/usfws_top10.total_count_by_year*100,2)

# usfws_top10

In [ ]:
# # creating subset called usfws next top 10 (11-20), grouping by year & common name
# usfws_next_top10 = birds_usfws.loc[birds_usfws.common_name.isin(['Turkey Vulture', 'Double-crested Cormorant', 'Blue Jay', 'Ring-billed Gull', 'Carolina Chickadee', 'Red-winged Blackbird', 'Eastern Bluebird', 'Pied-billed Grebe', 'Great Egret', 'Mourning Dove'])]
# usfws_next_top10 = usfws_next_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
# usfws_next_top10.columns = ['observation_year', 'common_name', 'count']

# # adding total count per year as a column
# total_birds = usfws_next_top10.groupby('observation_year').sum()[['count']]
# total_birds = total_birds.reset_index()
# total_birds.columns=['observation_year', 'total_count_by_year']
# # merging back
# usfws_next_top10 = pd.merge(usfws_next_top10, total_birds, on='observation_year', how='inner')

# # adding perc column
# usfws_next_top10['perc_total_sightings'] = round(usfws_next_top10['count']/usfws_next_top10.total_count_by_year*100,2)
# usfws_next_top10

### Unique observer id count

In [ ]:
# total number of unique birders 2007-2016
len(birds.observer_id.unique())

In [ ]:
# for all birds

# count of unique birders per year
birders_per_year = birds.groupby('observation_year')['observer_id'].unique().reset_index()
birders_per_year['count_birders'] = birders_per_year['observer_id'].str.len()

# getting count of sightings per year
birds_per_year = birds.groupby('observation_year').count()[['global_unique_identifier']].reset_index()
birds_per_year.columns = ['observation_year', 'count_birds']
birds_and_birders = pd.merge(birders_per_year, birds_per_year, on='observation_year', how='inner')
birds_and_birders['birds_per_birder'] = round(birds_and_birders.count_birds/birds_and_birders.count_birders,2)
birds_and_birders = birds_and_birders[['observation_year', 'count_birders', 'count_birds', 'birds_per_birder']]
birds_and_birders.head()

In [ ]:
birds_and_birders.to_csv('birds_per_birder.csv')

In [ ]:
# for IBA birds

# count of unique birders per year
birders_per_year = iba_birds.groupby('observation_year')['observer_id'].unique().reset_index()
birders_per_year['count_birders'] = birders_per_year['observer_id'].str.len()

# getting count of sightings per year
birds_per_year = iba_birds.groupby('observation_year').count()[['global_unique_identifier']].reset_index()
birds_per_year.columns = ['observation_year', 'count_birds']
birds_and_birders = pd.merge(birders_per_year, birds_per_year, on='observation_year', how='inner')
birds_and_birders['birds_per_birder'] = round(birds_and_birders.count_birds/birds_and_birders.count_birders,2)
iba_birds_and_birders = birds_and_birders[['observation_year', 'count_birders', 'count_birds', 'birds_per_birder']]
iba_birds_and_birders

In [ ]:
# for all nashville birds (no restriction)

all_nashville_birds = birds.loc[birds.county == 'Davidson']

# count of unique birders per year
birders_per_year = all_nashville_birds.groupby('observation_year')['observer_id'].unique().reset_index()
birders_per_year['count_birders'] = birders_per_year['observer_id'].str.len()

# getting count of sightings per year
birds_per_year = all_nashville_birds.groupby('observation_year').count()[['global_unique_identifier']].reset_index()
birds_per_year.columns = ['observation_year', 'count_birds']
birds_and_birders = pd.merge(birders_per_year, birds_per_year, on='observation_year', how='inner')
birds_and_birders['birds_per_birder'] = round(birds_and_birders.count_birds/birds_and_birders.count_birders,2)
all_nash_birds_and_birders = birds_and_birders[['observation_year', 'count_birders', 'count_birds', 'birds_per_birder']]
all_nash_birds_and_birders

In [ ]:
# for nashville birds (restricted to 3rd quartile)

# count of unique birders per year
birders_per_year = nashville_birds.groupby('observation_year')['observer_id'].unique().reset_index()
birders_per_year['count_birders'] = birders_per_year['observer_id'].str.len()

# getting count of sightings per year
birds_per_year = nashville_birds.groupby('observation_year').count()[['global_unique_identifier']].reset_index()
birds_per_year.columns = ['observation_year', 'count_birds']
birds_and_birders = pd.merge(birders_per_year, birds_per_year, on='observation_year', how='inner')
birds_and_birders['birds_per_birder'] = round(birds_and_birders.count_birds/birds_and_birders.count_birders,2)
nash_birds_and_birders = birds_and_birders[['observation_year', 'count_birders', 'count_birds', 'birds_per_birder']]
nash_birds_and_birders

#### list of sensitive species for map

In [ ]:
sensitive_species['geometry'] = sensitive_species.apply(lambda x: Point((float(x.long),
                                                                  float(x.lat))),
                                                 axis=1)

## Export CSVs:

In [ ]:
## TOP 10 LISTS/COUNTS

# iba top 10
iba_top10.to_csv('iba_top10.csv')

# iba next top 10
iba_next_top10.to_csv('iba_next_top10.csv')

# outside iba top 10
outside_iba_top10.to_csv('outside_iba_top10.csv')

# outside iba next top 10
outside_iba_next_top10.to_csv('outside_iba_next_top10.csv')

# nash birds count
nash_birds_count.to_csv('nash_birds_count.csv')

# birds of interest count
birds_of_interest_count.to_csv('birds_of_interest_count.csv')

## MISC

# birds by season
birds_by_season.to_csv('birds_by_season.csv')

# birds per birder
birds_and_birders.to_csv('birds_per_birder.csv')

# iba birds per birder
iba_birds_and_birders.to_csv('iba_birds_per_birder.csv')

# restricted nashville birds per birder
nash_birds_and_birders.to_csv('res_nash_birds_per_birder.csv')

# all nashville birds per birder
all_nash_birds_and_birders.to_csv('all_nash_birds_per_birder.csv')

# MAPS

# iba top 20 for map
iba_top20.to_csv('iba_top20_map.csv')

# outside iba top 20 for map
outside_iba_top20.to_csv('outside_iba_top20_map.csv')

# birds of interest for map
birds_of_interest.to_csv('birds_of_interest_map.csv')

# sensitive species list for map
sensitive_species.to_csv('sensitive_species_map.csv')

# nashville birds for map
nashville_birds.to_csv('nashville_birds_map.csv')

### attempting webscraping:

come back to later

In [ ]:
# come back to later!!
# download Selenium webdriver

In [ ]:
# # importing beautiful soup
# import requests
# from bs4 import BeautifulSoup as BS

In [ ]:
# # establishing connection
# URL = 'https://www.fws.gov/refuge/tennessee/map'
# response = requests.get(URL)

In [ ]:
# # checking connection
# response.status_code

In [ ]:
# soup = BS(response.text)

In [ ]:
# soup.findAll('path', attrs={'class':'leaflet-interactive'})